In [ ]:
!python train.py --epochs 1 --train-photo-path train_photo_test --dataset shinkai_test --batch 4 --init-epochs 4 --use_sn 

In [ ]:
!python script/video_to_images.py --video-path ./dataset/video/city.gif

In [ ]:
!python script/edge_smooth.py --dataset shinkai --image-size 256

In [ ]:
!python script/edge_smooth.py --dataset hayao --image-size 256

In [ ]:
!python script/edge_smooth.py --dataset paprika --image-size 256

In [24]:
from utils.common import load_checkpoint,load_weight
from modeling.anime_ganv2 import Generator
import os
def load_checkpoint(model, checkpoint_dir, posfix=''):
    path = os.path.join(checkpoint_dir, f'{model.name}{posfix}.pth')
    return load_weight(model, path)
G = Generator('shinkai').to('cpu')

start_e = load_checkpoint(G, 'checkpoint')
start_e

5